In [ ]:
!pip install numpy-financial

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy_financial as npf
import numpy as np # Import numpy for isnan check

# Function to calculate NPV (reusing the previous definition)
def npv(rate, cashflows):
    """
    rate: discount rate (example: 0.10 for 10%)
    cashflows: list of cash flows, where
               cashflows[0] is the initial investment (usually negative)
               cashflows[1], cashflows[2], ... are inflows in future years
    """
    total_value = 0
    for t, cash in enumerate(cashflows):
        present_value = cash / ((1 + rate) ** t)
        total_value += present_value
    return total_value

# Function to calculate IRR (reusing the previous definition)
def irr(cashflows):
    """
    cashflows: list of cash flows, where
               cashflows[0] is the initial investment (usually negative)
               cashflows[1], cashflows[2], ... are inflows in future years
    """
    try:
        return npf.irr(cashflows)
    except:
        return float('nan') # Return NaN if IRR cannot be calculated


# Create input widgets
cash_flows_input = widgets.Textarea(
    description='Cash Flows:',
    placeholder='-1000, 200, 300, 400, 500',
    disabled=False
)

discount_rate_input = widgets.FloatText(
    description='Discount Rate (for NPV):',
    value=0.10,
    disabled=False
)

hurdle_rate_input = widgets.FloatText(
    description='Hurdle Rate (for IRR):',
    value=0.15, # Example hurdle rate
    disabled=False
)

calculate_button = widgets.Button(description='Calculate')

# Output widget to display results
output = widgets.Output()

# Function to handle button click
def on_button_clicked(b):
    with output:
        clear_output()
        try:
            cash_flows = [float(x.strip()) for x in cash_flows_input.value.split(',')]
            discount_rate = discount_rate_input.value
            hurdle_rate = hurdle_rate_input.value

            npv_value = npv(discount_rate, cash_flows)
            irr_value = irr(cash_flows)

            print(f"Net Present Value (NPV): {npv_value:.2f}")
            if not np.isnan(irr_value):
                 print(f"Internal Rate of Return (IRR): {irr_value:.2%}")
            else:
                 print("Internal Rate of Return (IRR): Could not calculate")
            print(f"Hurdle Rate: {hurdle_rate:.2%}")


            print("\n--- Investment Decision Rule ---")
            if npv_value > 0:
                print("Based on NPV: Accept the investment.")
            elif npv_value < 0:
                print("Based on NPV: Reject the investment.")
            else:
                print("Based on NPV: The investment is neutral.")

            if not np.isnan(irr_value):
                if irr_value > hurdle_rate:
                    print(f"Based on IRR: Accept the investment (IRR > Hurdle Rate).")
                elif irr_value < hurdle_rate:
                    print(f"Based on IRR: Reject the investment (IRR < Hurdle Rate).")
                else:
                    print(f"Based on IRR: The investment is neutral (IRR = Hurdle Rate).")
            else:
                 print("Based on IRR: Cannot apply decision rule as IRR could not be calculated.")


        except ValueError:
            print("Invalid input. Please ensure cash flows are numbers separated by commas.")
        except Exception as e:
            print(f"An error occurred: {e}")


# Link button click to function
calculate_button.on_click(on_button_clicked)

# Display the widgets
display(cash_flows_input, discount_rate_input, hurdle_rate_input, calculate_button, output)

Textarea(value='', description='Cash Flows:', placeholder='-1000, 200, 300, 400, 500')

FloatText(value=0.1, description='Discount Rate (for NPV):')

FloatText(value=0.15, description='Hurdle Rate (for IRR):')

Button(description='Calculate', style=ButtonStyle())

Output()